In [172]:
# !pip install vectorbt

In [205]:
import pandas as pd
import numpy as np
import vectorbt as vbt

In [206]:
df = pd.read_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/polentovisk_2018_2021.csv', sep=';')
df

,Date,Tipo,Volume,Price,SL,TP,Saldo,Price Vol,Scaled Price Return,Retorno,Dummy
0,2018.01.02 16:57:00,sell,19,78310,77440,80340,1002206,"0,0001","-0,028285311",2206,1
1,2018.01.02 17:00:01,sell,19,78310,77440,80340,1004412,"0,0000","-0,032808644",2206,1
2,2018.01.04 16:57:00,sell,19,79045,78730,81630,1004499,"0,0094","0,454192378",87,1
3,2018.01.04 17:00:01,sell,19,79160,78730,81630,1005023,"0,0015","0,042680273",524,1
4,2018.01.08 11:34:21,buy,38,79400,79390,76490,1002747,"0,0030","0,124504574",-2276,0
...,...,...,...,...,...,...,...,...,...,...,...
741,2021.10.13 16:50:14,sell,38,114605,114205,117105,1367659,"0,0165","0,820855657",860,1
742,2021.10.15 16:57:00,sell,19,115740,114935,117835,1369618,"0,0099","0,481059923",1959,1
743,2021.10.15 17:00:01,sell,19,115675,114935,117835,1371330,"-0,0006","-0,061948649",1712,1
744,2021.10.28 11:23:01,sell,38,107475,107475,110375,1369130,"-0,0709","-3,710998149",-2200,0


In [207]:
for col in df.columns:
    if col == 'Price Vol' or col == 'Scaled Price Return':
        for row in range(len(df[col])):
            string = str(df[col][row])
            df[col][row] = float(string.replace(',', '.'))

<ipython-input-207-801c6725eb1b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][row] = float(string.replace(',', '.'))


In [208]:
df.rename(columns={'Date': 'date'}, inplace=True)
df['date'] = pd.to_datetime(df['date'])

In [209]:
df_win_2019 = pd.read_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/OHLC_1SWIN$N_2019_BMF_T.csv')
df_win_2020 = pd.read_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/OHLC_1SWIN$N_2020_BMF_T.csv')
df_win_2021 = pd.read_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/OHLC_1SWIN$N_2021_BMF_T.csv')

In [210]:
df_win_2019.rename(columns={'<DATE>_<TIME>': 'date'}, inplace=True)
df_win_2020.rename(columns={'<DATE>_<TIME>': 'date'}, inplace=True)
df_win_2021.rename(columns={'<DATE>_<TIME>': 'date'}, inplace=True)

df_win_2019['date'] = pd.to_datetime(df_win_2019['date'])
df_win_2020['date'] = pd.to_datetime(df_win_2020['date'])
df_win_2021['date'] = pd.to_datetime(df_win_2021['date'])

In [211]:
df_win_2019.set_index('date', drop=True, inplace=True)
df_win_2020.set_index('date', drop=True, inplace=True)
df_win_2021.set_index('date', drop=True, inplace=True)

df.set_index('date', drop=True, inplace=True)

In [212]:
merge_df_2019 = pd.merge(df, df_win_2019, how='inner', left_index=True, right_index=True)
merge_df_2020 = pd.merge(df, df_win_2020, how='inner', left_index=True, right_index=True)
merge_df_2021 = pd.merge(df, df_win_2021, how='inner', left_index=True, right_index=True)

In [213]:
merge_df_2019.rename(columns={'<OPEN>': 'Open', '<HIGH>': 'High', '<LOW>': 'Low', '<CLOSE>': 'Close', '<TICK>': 'Tick', '<VOL>': 'Vol'}, inplace=True)
merge_df_2020.rename(columns={'<OPEN>': 'Open', '<HIGH>': 'High', '<LOW>': 'Low', '<CLOSE>': 'Close', '<TICK>': 'Tick', '<VOL>': 'Vol'}, inplace=True)
merge_df_2021.rename(columns={'<OPEN>': 'Open', '<HIGH>': 'High', '<LOW>': 'Low', '<CLOSE>': 'Close', '<TICK>': 'Tick', '<VOL>': 'Vol'}, inplace=True)

In [214]:
dfs = [merge_df_2019, merge_df_2020, merge_df_2021]

In [215]:
merge_df = pd.concat(dfs)

In [216]:
returns = [0]

for i in range(1, len(merge_df['Close'])):
    returns.append((merge_df['Close'][i] - merge_df['Close'][i-1])/merge_df['Close'][i-1])
returns

merge_df['Returns'] = returns

In [217]:
scaled_returns = []

mean = merge_df['Returns'].mean()
std = merge_df['Returns'].std()

for i in range(len(merge_df['Returns'])):
    scaled_returns.append((merge_df['Returns'][0] - mean)/std)
    
merge_df['Scaled Returns'] = scaled_returns

In [218]:
from math import pi, sqrt, exp

proba_return = []

for i in range(len(merge_df['Scaled Returns'])):
    proba_return.append(1/(sqrt(2*pi)*exp(-0.5*merge_df['Scaled Returns'][i]*merge_df['Scaled Returns'][i])))
    
merge_df['Proba Returns'] = proba_return

In [219]:
mean_candle = []

for i in range(len(merge_df['Scaled Returns'])):
    mean_candle.append(merge_df['High'][i] - merge_df['Low'][i])
    
merge_df['Candle Size'] = mean_candle

In [220]:
merge_df['MM20'] = vbt.MA.run(merge_df['Candle Size'], 20).ma
merge_df['MM10'] = vbt.MA.run(merge_df['Candle Size'], 10).ma
merge_df['MM5'] = vbt.MA.run(merge_df['Candle Size'], 5).ma
merge_df['MM2'] = vbt.MA.run(merge_df['Candle Size'], 2).ma

In [221]:
merge_df['MM20'].fillna(merge_df['MM20'].median(), inplace=True)
merge_df['MM10'].fillna(merge_df['MM10'].median(), inplace=True)
merge_df['MM5'].fillna(merge_df['MM5'].median(), inplace=True)
merge_df['MM2'].fillna(merge_df['MM2'].median(), inplace=True)

In [222]:
merge_df

,Tipo,Volume,Price,SL,TP,Saldo,Price Vol,Scaled Price Return,Retorno,Dummy,...,Tick,Vol,Returns,Scaled Returns,Proba Returns,Candle Size,MM20,MM10,MM5,MM2
date,,,,,,,,,,,,,,,,,,,,,
2019-04-02 15:30:14,buy,38,95640,96035,93135,1093457,0.0059,0.27554,862,1,...,141,710,0.000000,-0.043876,0.399326,25.0,12.00,11.5,12.0,10.0
2019-04-02 15:53:14,sell,38,95315,95340,98240,1091047,-0.0034,-0.209129,-2410,0,...,32,108,-0.001046,-0.043876,0.399326,5.0,12.00,11.5,12.0,15.0
2019-04-03 14:43:08,sell,38,96075,96075,98975,1088827,0.008,0.380917,-2220,0,...,331,1016,0.008851,-0.043876,0.399326,15.0,12.00,11.5,12.0,10.0
2019-04-03 16:57:00,buy,9,94665,96035,93135,1090753,-0.0147,-0.794306,1926,1,...,31,65,-0.017028,-0.043876,0.399326,10.0,12.00,11.5,12.0,12.5
2019-04-03 17:00:01,buy,9,94670,96035,93135,1092670,0.0001,-0.030068,1917,1,...,151,330,0.000000,-0.043876,0.399326,15.0,12.00,11.5,14.0,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23 14:31:14,buy,38,129450,129435,126535,1338744,-0.0029,-0.182685,-2274,0,...,34,202,-0.003736,-0.043876,0.399326,5.0,13.25,11.0,11.0,7.5
2021-06-24 16:45:16,buy,38,130130,130115,127215,1336430,0.0053,0.239754,-2314,0,...,43,271,0.005682,-0.043876,0.399326,10.0,13.25,10.5,9.0,7.5
2021-06-25 16:57:00,buy,19,127720,129590,126690,1342446,-0.0185,-0.993755,6016,1,...,56,173,-0.018142,-0.043876,0.399326,10.0,13.50,11.0,8.0,10.0


In [223]:
merge_df.to_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/STRAT_POLENTOVISK_WIN.csv', index=True)